In [4]:
import requests
from bs4 import BeautifulSoup
import re
import time
from collections import defaultdict
import copy
import pandas as pd
from IPython.core.display import display, HTML
import pickle
import os

In [5]:
# vendor = 'Schmincke'
# line = 'Horadam-Aquarell'

In [6]:
# vendor = 'Winsor-and-Newton'
# line = 'Artists'

In [4]:
# vendor = 'Holbein'
# line = 
#'Irodori-Antique' 
#'Artists'

In [8]:
# vendor = 'Daniel-Smith'
# line = 'Extra-Fine' 


#'Primatek' 
#'Pearlescent' 
#'Iridescent' 
#'Interference' 
#'Extra-Fine-Sticks' 
#'Extra-Fine' 
#'Duochrome' 

In [9]:
vendor = 'Sennelier'
line = 'Aquarelle'

# For vendor_line, generate pickle

In [10]:
if vendor not in ['Sennelier']:
    base_url = f"http://www.art-paints.com/Paints/Watercolor/{vendor}/{line}/"
else:
    base_url = f"http://www.art-paints.com/Paints/Watercolor/{vendor}/"
    
url = base_url + f"{vendor}-{line}.html"
payload = {}
headers = {}
response = requests.request("GET", url, headers=headers, data=payload)

In [11]:
res = BeautifulSoup(response.text, "html.parser")
hrefs = res.find_all("a", href=True)

In [12]:
vendor_line = set(h["href"]
                  for h in hrefs
                  if 'http://' not in h["href"])
vendor_line = list(vendor_line)
vendor_line.sort()

In [13]:
len(vendor_line)

80

# For vendor_line, iterate over all available colors

In [30]:
CMYK_pattern = 'CMYK #'
vendor_line_colors = dict()

for link in vendor_line:
    tube_name = link.split("/")[0]
    response_ = requests.request("GET", base_url + link, headers=headers, data=payload)
    res_ = BeautifulSoup(response_.text, "html.parser")
    hrefs_ = res_.find_all("a", href=True)
    CMYK_value = res_.find(text=re.compile(CMYK_pattern)).findNext('strong').contents[0].get_text()

    related_colors = set(h["href"].split("/")[1]
                         for h in hrefs_
                         if '../' in h["href"] and vendor not in h["href"])
    vendor_line_colors[tube_name] = {"cmyk": CMYK_value, "related": related_colors}




with open(f'{vendor}_{line}_colors.pickle', 'wb') as handle:
    pickle.dump(vendor_line_colors, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Combined 

In [31]:

def load_vendor_line_colors_from_pickle(vendor, line):
    with open(f'{vendor}_{line}_colors.pickle', 'rb') as handle:
        vendor_line_colors = pickle.load(handle)

    vendor_line_colors_enhanced = copy.deepcopy(vendor_line_colors)


    for k, v in vendor_line_colors_enhanced.items():
        # new_val = {rc :  vendor_line_colors_enhanced[rc]["cmyk"] for rc in v["related"] }
        # new_val = [{rc : vendor_line_colors_enhanced[rc]["cmyk"]} for rc in v["related"]]
        new_val = [f'{rc}<br>{vendor_line_colors_enhanced[rc]["cmyk"]}' for rc in v["related"]]

        vendor_line_colors_enhanced[k]["related"] = new_val



    vendor_line_df = pd.DataFrame.from_dict(vendor_line_colors_enhanced).transpose()
    vendor_line_df.reset_index(inplace=True)
    vendor_line_df.rename({"index": "name"}, inplace=True, axis=1)
    df_related_colors = vendor_line_df["related"].apply(
        lambda x: pd.Series([elem for elem in x])
    )
    vendor_line_df.drop(columns=['related'], inplace=True)
    final = pd.merge(vendor_line_df, df_related_colors, left_index=True, right_index=True)


    final[["c", "m", "y", "k"]] = final["cmyk"].apply(lambda x: pd.Series(x.split(","))).astype(int)
    final['vendor'] = vendor
    final['line'] = line
    final.drop(columns=['cmyk'], inplace=True)
    return final

In [32]:
def combine_all_finals(all_vendor_line_pickles):
    combined = pd.DataFrame()
    for file in all_vendor_line_pickles:
        curr_vendor, curr_line, _ = file.split("_")
        combined = combined.append(load_vendor_line_colors_from_pickle(curr_vendor, curr_line))
    
    return combined
    

In [33]:
def get_similar_color(color_in_question, final, tolerance):
    cmyk_vals = final[["c", "m", "y", "k"]].loc[final.name == color_in_question].values
    c_val = cmyk_vals.item(0)
    m_val = cmyk_vals.item(1)
    y_val = cmyk_vals.item(2)
    k_val = cmyk_vals.item(3)
    return final.loc[(abs(final.c - c_val) <= tolerance) &
                     (abs(final.m - m_val) <= tolerance) &
                     (abs(final.y - y_val) <= tolerance) &
                     (abs(final.k - k_val) <= tolerance)].copy()


def pretty_display(final):
    display(final.style.set_properties(**{
        'text-align': 'center',
        'white-space': 'pre-wrap',
        'width': '150px'
    }))
    
def find_nearest_color(comma_delimited_cmyk, final, tolerance):
    cmyk = comma_delimited_cmyk.split(", ")
    cmyk = [int(elem) for elem in cmyk]
    return final.loc[(abs(final.c - cmyk[0]) <= tolerance) &
                      (abs(final.m - cmyk[1]) <= tolerance) &
                      (abs(final.y - cmyk[2]) <= tolerance) &
                      (abs(final.k - cmyk[3]) <= tolerance)]

In [84]:
def find_nearest_color_v2(comma_delimited_cmyk, final, max_distance, top_n=5):
    cmyk = comma_delimited_cmyk.split(", ")
    cmyk = [int(elem) for elem in cmyk]
    final['distance'] = ((final['c'] - cmyk[0])**2 + \
                        (final['m'] - cmyk[1])**2 + \
                        (final['y'] - cmyk[2])**2 + \
                        (final['k'] - cmyk[3])**2)**0.5
    final['distance'] = final['distance'].astype(int)
    return final.sort_values(by=['distance']).head(top_n)
    

In [85]:
all_vendor_line_pickles = [f for f in os.listdir('.') if os.path.isfile(f) and f.endswith("_colors.pickle")]

In [86]:
combined = combine_all_finals(all_vendor_line_pickles)
combined.reset_index(drop=True, inplace=True)

In [87]:
combined.tail()

,name,0,1,2,3,4,5,6,c,m,y,k,vendor,line
704,Winsor-Violet-Dioxazine,"Cobalt-Violet<br>27, 59, 0, 0","Perylene-Violet<br>54, 77, 62, 67","Caput-Mortuum-Violet<br>50, 66, 56, 32","Neutral-Tint<br>75, 63, 59, 54","Mars-Black<br>70, 67, 66, 78","Manganese-Blue-Hue<br>70, 17, 0, 0","Ultramarine-Violet<br>58, 63, 2, 0",75,66,44,29,Winsor-and-Newton,Artists
705,Winsor-Yellow-Deep,"Cadmium-Yellow<br>0, 38, 88, 0","Naples-Yellow-Deep<br>0, 27, 75, 0","Turners-Yellow<br>2, 12, 86, 0","New-Gamboge<br>2, 17, 94, 0","Aureolin<br>3, 8, 99, 0","Yellow-Ochre<br>2, 39, 78, 0","Cadmium-Yellow-Pale<br>1, 12, 95, 0",0,22,89,0,Winsor-and-Newton,Artists
706,Winsor-Yellow,"Cadmium-Yellow-Pale<br>1, 12, 95, 0","Winsor-Lemon<br>4, 0, 93, 0","Lemon-Yellow-Nickel-Titanate<br>1, 8, 79, 0","Cadmium-Lemon<br>4, 0, 95, 0","Aureolin<br>3, 8, 99, 0","Lemon-Yellow-Deep<br>6, 1, 96, 0","Transparent-Yellow<br>3, 1, 96, 0",2,5,98,0,Winsor-and-Newton,Artists
707,Yellow-Ochre-Light,"Cadmium-Yellow<br>0, 38, 88, 0","Cadmium-Yellow-Deep<br>0, 51, 85, 0","Cadmium-Orange<br>0, 59, 81, 0","Naples-Yellow-Deep<br>0, 27, 75, 0","Winsor-Orange<br>0, 54, 71, 0","Indian-Yellow<br>0, 44, 99, 0","Yellow-Ochre<br>2, 39, 78, 0",7,42,91,0,Winsor-and-Newton,Artists
708,Yellow-Ochre,"Yellow-Ochre-Light<br>7, 42, 91, 0","Cadmium-Yellow<br>0, 38, 88, 0","Cadmium-Yellow-Deep<br>0, 51, 85, 0","Naples-Yellow-Deep<br>0, 27, 75, 0","Winsor-Yellow-Deep<br>0, 22, 89, 0","Indian-Yellow<br>0, 44, 99, 0","New-Gamboge<br>2, 17, 94, 0",2,39,78,0,Winsor-and-Newton,Artists


# Query

In [88]:
pretty_display(get_similar_color('Opera-Pink', combined, 5))

,name,0,1,2,3,4,5,6,c,m,y,k,vendor,line
124,Opera-Pink,"Potters-Pink-Pinkcolor36, 87, 54, 24","Quinacridone-Gold37, 53, 55, 11","Rose-Madder-Genuine0, 62, 43, 0","Indian-Red50, 63, 70, 43","Lunar-Red-Rock45, 74, 68, 52","Quinacridone-Fuchsia24, 91, 61, 11","Yavapai-Genuine45, 65, 79, 46",1,89,16,0,Daniel-Smith,Extra-Fine
481,Magenta,"Cobalt-Turquoise70, 2, 31, 0","Helio-Cerulean90, 29, 5, 0","Helio-Turquoise96, 35, 29, 5","Cobalt-Cerulean90, 36, 20, 4","Purple-Magenta13, 93, 8, 0","Mauve82, 91, 15, 4","Brilliant-Purple30, 100, 35, 0",6,93,17,0,Schmincke,Horadam-Aquarell


In [89]:
Holbein__Artists__Brown_Madder = '19, 84, 81, 7'
pretty_display(find_nearest_color(Holbein__Artists__Brown_Madder, combined, 6))

,name,0,1,2,3,4,5,6,c,m,y,k,vendor,line
28,Anthraquinoid-Red,"Permanent-Red15, 92, 87, 4","Pyrrol-Red15, 92, 84, 4","Cadmium-Red-Deep20, 90, 80, 10","Quinacridone-Rose28, 86, 68, 19","Quinacridone-Red20, 91, 81, 10","Perylene-Red23, 84, 71, 11","Perylene-Scarlet26, 84, 77, 18",22,90,77,12,Daniel-Smith,Extra-Fine
45,Cadmium-Red-Deep,"Permanent-Red15, 92, 87, 4","Organic-Vermilion9, 89, 83, 1","Pyrrol-Red15, 92, 84, 4","Cadmium-Red-Scarlet8, 84, 81, 1","Quinacridone-Red20, 91, 81, 10","Perylene-Red23, 84, 71, 11","Anthraquinoid-Red22, 90, 77, 12",20,90,80,10,Daniel-Smith,Extra-Fine
168,Quinacridone-Sienna,"Quinacridone-Pink26, 100, 92, 25","Terre-Ercolano12, 75, 68, 1","Transparent-Pyrrol-Orange21, 90, 93, 12","Minnesote-Pipestone23, 74, 73, 10","Italian-Venetian-Red20, 78, 64, 5","Permanent-Alizarin-Crimson24, 96, 85, 17","Pompeii-Red23, 71, 72, 9",13,79,79,2,Daniel-Smith,Extra-Fine
275,Brown-Madder,"Viridian-Hue89, 49, 74, 56","Ultramarine-Light94, 83, 0, 0","Viridian86, 45, 71, 41","Crimson-Lake24, 100, 100, 21","Carmine23, 99, 100, 16","Ultramarine-Deep96, 87, 0, 0","Raw-Sienna26, 55, 100, 9",19,84,81,7,Holbein,Artists
545,Cadmium-Red-Purple,"Bright-Red5, 87, 82, 1","Cadmium-Red-Orange15, 71, 87, 3","Venetian-Red20, 72, 78, 6","Crimson-Lake8, 95, 67, 1","Chinese-Orange14, 69, 85, 2","Red-Brown21, 73, 71, 7","Cadmium-Red-Light0, 85, 82, 0",22,81,75,10,Sennelier,Aquarelle
553,Chinese-Vermilion,"Bright-Red5, 87, 82, 1","Quinacridone-Red-Orange8, 87, 58, 1","Crimson-Lake8, 95, 67, 1","Carmine-Genuine15, 88, 52, 1","Cadmium-Red-Purple22, 81, 75, 10","Cadmium-Red-Light0, 85, 82, 0",nan,14,85,84,3,Sennelier,Aquarelle


In [90]:
pretty_display(find_nearest_color_v2(Holbein__Artists__Brown_Madder, combined, 6, 10))

,name,0,1,2,3,4,5,6,c,m,y,k,vendor,line,distance
275,Brown-Madder,"Viridian-Hue89, 49, 74, 56","Ultramarine-Light94, 83, 0, 0","Viridian86, 45, 71, 41","Crimson-Lake24, 100, 100, 21","Carmine23, 99, 100, 16","Ultramarine-Deep96, 87, 0, 0","Raw-Sienna26, 55, 100, 9",19,84,81,7,Holbein,Artists,0
45,Cadmium-Red-Deep,"Permanent-Red15, 92, 87, 4","Organic-Vermilion9, 89, 83, 1","Pyrrol-Red15, 92, 84, 4","Cadmium-Red-Scarlet8, 84, 81, 1","Quinacridone-Red20, 91, 81, 10","Perylene-Red23, 84, 71, 11","Anthraquinoid-Red22, 90, 77, 12",20,90,80,10,Daniel-Smith,Extra-Fine,6
166,Quinacridone-Red,"Permanent-Red15, 92, 87, 4","Pyrrol-Red15, 92, 84, 4","Organic-Vermilion9, 89, 83, 1","Cadmium-Red-Deep20, 90, 80, 10","Perylene-Red23, 84, 71, 11","Anthraquinoid-Red22, 90, 77, 12","Perylene-Scarlet26, 84, 77, 18",20,91,81,10,Daniel-Smith,Extra-Fine,7
545,Cadmium-Red-Purple,"Bright-Red5, 87, 82, 1","Cadmium-Red-Orange15, 71, 87, 3","Venetian-Red20, 72, 78, 6","Crimson-Lake8, 95, 67, 1","Chinese-Orange14, 69, 85, 2","Red-Brown21, 73, 71, 7","Cadmium-Red-Light0, 85, 82, 0",22,81,75,10,Sennelier,Aquarelle,7
553,Chinese-Vermilion,"Bright-Red5, 87, 82, 1","Quinacridone-Red-Orange8, 87, 58, 1","Crimson-Lake8, 95, 67, 1","Carmine-Genuine15, 88, 52, 1","Cadmium-Red-Purple22, 81, 75, 10","Cadmium-Red-Light0, 85, 82, 0",nan,14,85,84,3,Sennelier,Aquarelle,7
28,Anthraquinoid-Red,"Permanent-Red15, 92, 87, 4","Pyrrol-Red15, 92, 84, 4","Cadmium-Red-Deep20, 90, 80, 10","Quinacridone-Rose28, 86, 68, 19","Quinacridone-Red20, 91, 81, 10","Perylene-Red23, 84, 71, 11","Perylene-Scarlet26, 84, 77, 18",22,90,77,12,Daniel-Smith,Extra-Fine,9
168,Quinacridone-Sienna,"Quinacridone-Pink26, 100, 92, 25","Terre-Ercolano12, 75, 68, 1","Transparent-Pyrrol-Orange21, 90, 93, 12","Minnesote-Pipestone23, 74, 73, 10","Italian-Venetian-Red20, 78, 64, 5","Permanent-Alizarin-Crimson24, 96, 85, 17","Pompeii-Red23, 71, 72, 9",13,79,79,2,Daniel-Smith,Extra-Fine,9
156,Pyrrol-Red,"Quinacridone-Coral9, 84, 76, 1","Permanent-Red15, 92, 87, 4","Organic-Vermilion9, 89, 83, 1","Cadmium-Red-Scarlet8, 84, 81, 1","Cadmium-Red-Deep20, 90, 80, 10","Quinacridone-Red20, 91, 81, 10","Anthraquinoid-Red22, 90, 77, 12",15,92,84,4,Daniel-Smith,Extra-Fine,9
693,Venetian-Red,"Brown-Madder24, 81, 66, 11","Permanent-Carmine13, 90, 61, 1","Indian-Red41, 70, 57, 26","Permanent-Alizarin-Crimson15, 83, 56, 2","Light-Red30, 73, 66, 18","Winsor-Red-Deep23, 100, 84, 15","Alizarin-Crimson20, 89, 69, 7",19,76,74,6,Winsor-and-Newton,Artists,10
139,Perylene-Red,"Pyrrol-Red15, 92, 84, 4","Cadmium-Red-Deep20, 90, 80, 10","Quinacridone-Rose28, 86, 68, 19","Quinacridone-Red20, 91, 81, 10","Carmine30, 83, 71, 25","Anthraquinoid-Red22, 90, 77, 12","Perylene-Scarlet26, 84, 77, 18",23,84,71,11,Daniel-Smith,Extra-Fine,11


In [91]:
# combined[combined.vendor == 'Holbein']

In [75]:
# pretty_display(find_nearest_color_v2(Holbein__Artists__Brown_Madder, query_by_vendor, 6))